In [49]:
import pyodbc
import tensorflow as tf
from joblib import load
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [2]:

# Connect to MSSQL database

driver= '{ODBC Driver 17 for SQL Server}'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=AnimeInsight;UID=sa;PWD=12345678;Trusted_Connection=yes')

cursor = connection.cursor()


In [3]:

df = pd.read_sql('SELECT * FROM anime_metadata', connection)


C:\Users\aunno\AppData\Local\Temp\ipykernel_4892\4234207955.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM anime_metadata', connection)


In [48]:
connection.close()

In [4]:
df

,anime_id,Name,English_name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Age_Rating,Rank,Scored_By,Image_URL
0,1,Cowboy Bebop,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,914193.0,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",None,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,206248.0,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,356739.0,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",summer 2002,Finished Airing,"Bandai Visual, Dentsu, Victor Entertainment, T...","Funimation, Bandai Entertainment",Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,42829.0,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",fall 2004,Finished Airing,"TV Tokyo, Dentsu",Illumitoon Entertainment,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,6413.0,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,None,NaN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",None,Not yet aired,None,None,None,Web manga,None,PG-13 - Teens 13 or older,NaN,NaN,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,NaN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",None,Not yet aired,None,None,None,Web novel,None,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,NaN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",None,Finished Airing,None,None,None,Web novel,None,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,None,NaN,None,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",None,Finished Airing,None,None,None,Original,3 min,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1931/...


In [5]:

# Load your TensorFlow model
model = tf.keras.models.load_model('models/anime_model.keras')


In [7]:
mean_score_scaler=load('preprocessing/mean_score_scaler.pkl')

rating_scaler=load('preprocessing/rating_scaler.pkl')

score_scaler=load('preprocessing/score_scaler.pkl')

type_encoder=load('preprocessing/Type_encoder.pkl')


gender_encoder=load('preprocessing/gender_encoder.pkl')

user_input_shape=6

anime_input_shape=8

studio_max_length = 10

genre_max_length=9

source_max_length=1

In [8]:
@tf.keras.utils.register_keras_serializable()
def split_func(input_str):
  return tf.strings.split(input_str, sep=", ")

In [9]:
studio_vectorize_layer_model=tf.keras.models.load_model('preprocessing/studio_vectorize_layer_model')
studio_vectorize_layer = studio_vectorize_layer_model.layers[0]

num_studios=len(studio_vectorize_layer.get_vocabulary())
num_studios

914

In [10]:
genre_vectorize_layer_model=tf.keras.models.load_model('preprocessing/genre_vectorize_layer_model')
genre_vectorize_layer = genre_vectorize_layer_model.layers[0]

num_genres=len(genre_vectorize_layer.get_vocabulary())
num_genres

23

In [11]:
source_vectorize_layer_model=tf.keras.models.load_model('preprocessing/source_vectorize_layer_model')
source_vectorize_layer = source_vectorize_layer_model.layers[0]


num_sources=len(source_vectorize_layer.get_vocabulary())
num_sources

19

In [12]:
df.drop(['English_name',"Name",'Synopsis','Age_Rating','Rank','Scored_By','Status','Premiered','Episodes','Duration','Licensors','Producers' , 'Aired','Image_URL'],axis=1,inplace=True)

In [16]:
df.fillna({'Score':df['Score'].mean()},inplace=True)

In [18]:
df.loc[:,'Score']=score_scaler.transform(df[['Score']]).astype(np.float32)

c:\Users\aunno\.conda\envs\db-project\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


In [19]:
encoded_Types = type_encoder.transform(df[['Type']])  # Fit and transform the column
encoded_df = pd.DataFrame(encoded_Types.toarray(), columns=type_encoder.categories_[0],index=df.index)  # Create a DataFrame from the encoded columns

df=pd.concat([df, encoded_df], axis=1)  # Concatenate the original DataFrame with the encoded DataFrame
df.drop(['Type'],axis=1,inplace=True)

In [22]:
df.fillna({'Studios':'','Genres':'','Source':''},inplace=True)

In [23]:
df

,anime_id,Score,Genres,Studios,Source,Movie,Music,ONA,OVA,Special,TV,UNKNOWN
0,1,2.657709,"Action, Award Winning, Sci-Fi",Sunrise,Original,0,0,0,0,0,1,0
1,5,2.225084,"Action, Sci-Fi",Bones,Original,1,0,0,0,0,0,0
2,6,2.039589,"Action, Adventure, Sci-Fi",Madhouse,Manga,0,0,0,0,0,1,0
3,7,0.936342,"Action, Drama, Mystery, Supernatural",Sunrise,Original,0,0,0,0,0,1,0
4,8,0.591776,"Adventure, Fantasy, Supernatural",Toei Animation,Manga,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,-0.019334,"Comedy, Fantasy, Slice of Life",,Web manga,0,0,1,0,0,0,0
24901,55732,-0.019334,"Action, Adventure, Fantasy",,Web novel,0,0,1,0,0,0,0
24902,55733,-0.019334,"Action, Adventure, Fantasy, Sci-Fi",,Web novel,0,0,1,0,0,0,0
24903,55734,-0.019334,,,Original,0,1,0,0,0,0,0


In [24]:
sequences=studio_vectorize_layer(df.loc[:,'Studios'])
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=studio_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(studio_max_length):
    df[f'Studio_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Studios', axis=1, inplace=True)

In [25]:
sequences=genre_vectorize_layer(df.loc[:,'Genres'])

padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=genre_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(genre_max_length):
    df[f'Genre_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Genres', axis=1, inplace=True)

In [26]:
sequences=source_vectorize_layer(df.loc[:,'Source'])

padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=source_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(source_max_length):
    df[f'Source_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Source', axis=1, inplace=True)

In [27]:
anime_columns=df.iloc[:,1:1+anime_input_shape].to_numpy(dtype='float32')
anime_studio_embedding_columns = df.iloc[:,1+anime_input_shape:1+anime_input_shape+studio_max_length].to_numpy()
anime_genre_embedding_columns= df.iloc[:,1+anime_input_shape+studio_max_length:1+anime_input_shape+studio_max_length+genre_max_length].to_numpy()
anime_source_embedding_columns = df.iloc[:,-1:].to_numpy()

In [34]:
input={'anime_inputs':anime_columns,'anime_genre_embeddings_inputs':anime_genre_embedding_columns,'anime_studio_embeddings_inputs':anime_studio_embedding_columns,'anime_source_embeddings_inputs':anime_source_embedding_columns}

In [36]:
output=model(input)

In [40]:
output_df = pd.DataFrame(output,columns=['tensor_1', 'tensor_2', 'tensor_3', 'tensor_4', 'tensor_5', 'tensor_6', 'tensor_7', 'tensor_8', 'tensor_9', 'tensor_10', 'tensor_11', 'tensor_12', 'tensor_13', 'tensor_14', 'tensor_15', 'tensor_16'])

In [42]:
final_df =pd.concat([df['anime_id'], output_df], axis=1)

In [45]:
final_df.rename(columns={'anime_id':'AnimeID'},inplace=True)

In [46]:
final_df

,AnimeID,tensor_1,tensor_2,tensor_3,tensor_4,tensor_5,tensor_6,tensor_7,tensor_8,tensor_9,tensor_10,tensor_11,tensor_12,tensor_13,tensor_14,tensor_15,tensor_16
0,1,0.0,3.081308,0.001305,0.000295,0.0,1.479221,1.532943,0.0,0.0,0.0,0.0,0.0,0.0,2.725895,0.0,0.007029
1,5,0.0,2.989005,0.001305,0.000295,0.0,1.446133,1.493776,0.0,0.0,0.0,0.0,0.0,0.0,2.645122,0.0,0.007029
2,6,0.0,2.933825,0.001305,0.000295,0.0,1.426352,1.470361,0.0,0.0,0.0,0.0,0.0,0.0,2.596834,0.0,0.007029
3,7,0.0,2.737314,0.001305,0.000295,0.0,1.355908,1.386976,0.0,0.0,0.0,0.0,0.0,0.0,2.424869,0.0,0.007029
4,8,0.0,2.613059,0.001305,0.000295,0.0,1.311365,1.334251,0.0,0.0,0.0,0.0,0.0,0.0,2.316134,0.0,0.007029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,0.0,2.566413,0.001305,0.000295,0.0,1.294644,1.314458,0.0,0.0,0.0,0.0,0.0,0.0,2.275316,0.0,0.007029
24901,55732,0.0,2.554152,0.001305,0.000295,0.0,1.290249,1.309255,0.0,0.0,0.0,0.0,0.0,0.0,2.264586,0.0,0.007029
24902,55733,0.0,2.561262,0.001305,0.000295,0.0,1.292798,1.312272,0.0,0.0,0.0,0.0,0.0,0.0,2.270808,0.0,0.007029
24903,55734,0.0,2.543753,0.001305,0.000295,0.0,1.286521,1.304842,0.0,0.0,0.0,0.0,0.0,0.0,2.255486,0.0,0.007029


In [52]:
'DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=AnimeInsight;UID=sa;PWD=12345678;Trusted_Connection=yes'
connect_string = urllib.parse.quote_plus(f'DRIVER={{ODBC Driver 17 for SQL Server}};Server=localhost;Database=AnimeInsight;UID=sa;PWD=12345678;Trusted_Connection=yes')
engine = sqlalchemy.create_engine(f'mssql+pyodbc:///?odbc_connect={connect_string}', fast_executemany=True)

with engine.connect() as connection:
  final_df.to_sql('anime_tensors', connection, index=False, if_exists='append')